In [3]:
import os
import sys
import numpy as np
from PIL import Image
from tqdm import tqdm

### Functions for BundleFusion [dataset](http://graphics.stanford.edu/projects/bundlefusion/)

In [4]:
# rewritting sensorData.h

def make_matrix(data):
    return np.array([np.array(d, dtype=np.float32) for d in data[1:]]).reshape((4, 4))

def read_info(filename):
    header_data = {}
    with open(filename) as file:
        for line in file:
            print(line)
            tmp = line.strip().split('=')
            if 'Intrinsic' in tmp[0] or 'Extrinsic' in tmp[0]:
                header_data[tmp[0].rstrip()] = make_matrix(tmp[-1].rstrip().split(' '))
            else:
                print(tmp[-1])
                header_data[tmp[0].rstrip()] = tmp[-1].strip()
                    
    return header_data
  
def read_image(filename):
    im = Image.open(filename)
    return np.asarray(im)
    
def read_frames(folder, num_frames):
    frames = {'m_colorCompressed': [], 'm_depthCompressed': [], 'm_cameraToWorld': [],
              'm_timeStampColor': [], 'm_timeStampDepth': [], 
              'm_colorSizeBytes': [], 'm_depthSizeBytes': []}
    frames['num_frames'] = num_frames
    for i in range(int(num_frames)):
        frames['m_colorCompressed'].append(read_image(folder + f'/frame-{i:06}.color.jpg'))
        frames['m_timeStampColor'].append(i)
        frames['m_depthCompressed'].append(read_image(folder + f'/frame-{i:06}.depth.png'))
        frames['m_timeStampDepth'].append(i)
        
        frames['m_cameraToWorld'].append(np.loadtxt(folder + f'/frame-{i:06}.pose.txt'))
        h, w, c = frames['m_colorCompressed'][-1].shape
        frames['m_colorSizeBytes'].append(h*w*c) # uint8 * h * w * c 
        h, w = frames['m_depthCompressed'][-1].shape
        frames['m_depthSizeBytes'].append(2*h*w) # uint16 * h * w
    
    return frames

def writeHeaderToFile(out, data):
    # out.write(m_versionNumber, sizeof(unsigned int));
    # #define M_SENSOR_DATA_VERSION 4 (line 800)
    np.array(data['m_versionNumber'], dtype=np.uint32).tofile(out)
    print('m_versionNumber', data['m_versionNumber'])
    print('len(data sensor name)', len(data['m_sensorName'])) 
    np.array([len(data['m_sensorName'])], dtype=np.uint64).tofile(out)
    print('sensor name', data['m_sensorName'].strip())
    out.write(data['m_sensorName'].encode())
    # out.write((const char*)&m_extrinsic, sizeof(mat4f));
    print(data['m_calibrationColorIntrinsic'])
    data['m_calibrationColorIntrinsic'].tofile(out)
    print(data['m_calibrationColorExtrinsic'])
    data['m_calibrationColorExtrinsic'].tofile(out)
    print(data['m_calibrationDepthIntrinsic'])
    data['m_calibrationDepthIntrinsic'].tofile(out)
    print(data['m_calibrationDepthExtrinsic'])
    data['m_calibrationDepthExtrinsic'].tofile(out)
    
#     out.write((const char*)&m_colorCompressionType, sizeof(COMPRESSION_TYPE_COLOR));
#     out.write((const char*)&m_depthCompressionType, sizeof(COMPRESSION_TYPE_DEPTH));
#     m_colorCompressionType, m_depthCompressionType = TYPE_COLOR_UNKNOWN; TYPE_COLOR_UNKNOWN = -1;
    
    # compression type is temporarily set as UNKNOWN 
    np.array([0], dtype=np.int32).tofile(out)
    np.array([0], dtype=np.int32).tofile(out)
    
    np.array(data['m_colorWidth'], dtype=np.uint32).tofile(out)
    np.array(data['m_colorHeight'], dtype=np.uint32).tofile(out)
    np.array(data['m_depthWidth'], dtype=np.uint32).tofile(out)
    np.array(data['m_depthHeight'], dtype=np.uint32).tofile(out)
    np.array(data['m_depthShift'], dtype=np.float32).tofile(out)
    print('depth shift', data['m_depthShift'])

def writeRGBFramesToFile(out, rgbdframes):

    for i in tqdm(range(int(rgbdframes['num_frames']))):
        np.array(rgbdframes['m_cameraToWorld'][i], dtype=np.float32).tofile(out)
        np.array(rgbdframes['m_timeStampColor'][i], dtype=np.uint64).tofile(out)
        np.array(rgbdframes['m_timeStampDepth'][i], dtype=np.uint64).tofile(out)
        np.array(rgbdframes['m_colorSizeBytes'][i], dtype=np.uint64).tofile(out)
        np.array(rgbdframes['m_depthSizeBytes'][i], dtype=np.uint64).tofile(out)
        
        rgbdframes['m_colorCompressed'][i].tofile(out)
        np.array(rgbdframes['m_depthCompressed'][i], dtype=np.uint16).tofile(out)
        #np.array(rgbdframes['m_colorCompressed'][i], dtype=np.ubyte).tofile(out)
        # if i > 100:
        #     break

def writeIMUFramesToFile(out, data):
    '''
        write the following data to the .sens file: (the order matters)
            - number of frames
            - rotationRate
            - acceleration
            - magneticField
            - attitude
            - gravity
            - timeStamp 
        default data:
            num_frames = number of rgbd images in the folder
            IMUFrame() {
                    rotationRate = vec3d(0.0);
                    acceleration = vec3d(0.0);
                    magneticField = vec3d(0.0);
                    attitude = vec3d(0.0);
                    gravity = vec3d(0.0);
                    timeStamp = 0;
                }
            
    '''
    for key in data.keys():
        print(f'writting {key} to file, value = {data[key]}')
        if 'timeStamp' in key:
            np.array(data[key], dtype=np.int64).tofile(out)
        else:
            np.array(data[key], dtype=np.float).tofile(out)
        
def saveToFile(data_dir, filename_out, dir_out='C:/Users/unid0/Downloads/BundleFusion-master/BundleFusion-master/FriedLiver/x64/data'):
    #std::ofstream out(filename, std::ios::binary);
    out = open(f'{dir_out}/{filename_out}.sens', 'wb')

    header_filename = data_dir + '/info.txt'
    print('reading header...')
    data_header = read_info(header_filename)
    print('done!')
    print('reading imu file...')
    imu_filename = data_dir + '/imu.txt'
    imu_data = read_info(imu_filename)
    print('done!')
    #data_header['m_frames.size'] = 100
    imu_data['num_frames'] = data_header['m_frames.size']
    print('reading frames...')
    rgbdframes = read_frames(data_dir, data_header['m_frames.size'])#read rgbframes from data dir
    print('done!')
    
    print('writting header...')
    writeHeaderToFile(out, data_header)
    print('done!')
    print('writting rgbd frames...')
    np.array(int(data_header['m_frames.size']), np.int64).tofile(out)

    writeRGBFramesToFile(out, rgbdframes)
    print('done!')
    print('writting imu frames...')
    writeIMUFramesToFile(out, imu_data)
    print('done!')

In [80]:
saveToFile('./apt0', 'apt0_new')

reading header...
m_versionNumber = 4

 4
m_sensorName = StructureSensor

 StructureSensor
m_colorWidth = 640

 640
m_colorHeight = 480

 480
m_depthWidth = 640

 640
m_depthHeight = 480

 480
m_depthShift = 1000

 1000
m_calibrationColorIntrinsic = 582.871 0 320 0 0 582.871 240 0 0 0 1 0 0 0 0 1 

m_calibrationColorExtrinsic = 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 

m_calibrationDepthIntrinsic = 583 0 320 0 0 583 240 0 0 0 1 0 0 0 0 1 

m_calibrationDepthExtrinsic = 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 

m_frames.size = 8560

 8560
done!
reading imu file...
rotationRate = 0.0

 0.0
acceleration = 0.0

 0.0
magneticField = 0.0

 0.0
attitude = 0.0

 0.0
gravity = 0.0

 0.0
timeStamp = 0
 0
done!
reading frames...
done!
writting header...
m_versionNumber 4
len(data sensor name) 15
senosor name StructureSensor
[[582.871   0.    320.      0.   ]
 [  0.    582.871 240.      0.   ]
 [  0.      0.      1.      0.   ]
 [  0.      0.      0.      1.   ]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 

  1%|          | 89/8560 [00:00<00:09, 883.55it/s]

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
BYTES 921600 614400
[[ 9.99998e-01  1.74498e-03 -8.95149e-04 -6.79164e-06]
 [-1.74426e-03  9.99998e-01  7.97849e-04  8.22270e-04]
 [ 8.96555e-04 -7.96269e-04  9.99999e-01 -8.69697e-06]
 [ 0.00000e+00  0.00000e+00  0.00000e+00  1.00000e+00]]
BYTES 921600 614400
[[ 9.99994e-01  3.35404e-03 -6.88818e-04  4.50614e-05]
 [-3.35259e-03  9.99992e-01  2.10069e-03  2.10904e-03]
 [ 6.95841e-04 -2.09837e-03  9.99998e-01  8.12693e-04]
 [ 0.00000e+00  0.00000e+00  0.00000e+00  1.00000e+00]]
BYTES 921600 614400
[[ 9.99940e-01  1.10020e-02  8.58847e-05 -2.03865e-03]
 [-1.10023e-02  9.99881e-01  1.08288e-02  4.77726e-03]
 [ 3.31900e-05 -1.08291e-02  9.99941e-01  1.84944e-03]
 [ 0.00000e+00  0.00000e+00  0.00000e+00  1.00000e+00]]
BYTES 921600 614400
[[ 9.99946e-01  1.03698e-02  2.07547e-04 -2.61505e-03]
 [-1.03715e-02  9.99879e-01  1.16153e-02  4.64777e-03]
 [-8.71570e-05 -1.16169e-02  9.99933e-01  1.80929e-03]
 [ 0.00000e+00  0.00000e+00  0.

  3%|▎         | 278/8560 [00:00<00:08, 926.51it/s]

[[ 0.871663   0.331185  -0.361275  -0.0571494]
 [-0.385635   0.918394  -0.0885348 -0.37056  ]
 [ 0.302471   0.216493   0.928247   0.26475  ]
 [ 0.         0.         0.         1.       ]]
BYTES 921600 614400
[[ 0.870312   0.332842  -0.363006  -0.0618397]
 [-0.388125   0.917251  -0.0895013 -0.371573 ]
 [ 0.303178   0.218786   0.927478   0.265558 ]
 [ 0.         0.         0.         1.       ]]
BYTES 921600 614400
[[ 0.867856   0.334742  -0.367116  -0.0712036]
 [-0.392081   0.915289  -0.0922973 -0.374776 ]
 [ 0.305121   0.22404    0.925585   0.26714  ]
 [ 0.         0.         0.         1.       ]]
BYTES 921600 614400
[[ 0.866597   0.334768  -0.370054  -0.0761612]
 [-0.393316   0.91462   -0.093665  -0.376593 ]
 [ 0.307103   0.226718   0.924277   0.268068 ]
 [ 0.         0.         0.         1.       ]]
BYTES 921600 614400
[[ 0.862991   0.337479  -0.375972  -0.0849566]
 [-0.398113   0.912424  -0.0948051 -0.378605 ]
 [ 0.311051   0.231495   0.921769   0.271105 ]
 [ 0.         0.       

...

[[-0.746827    0.337564   -0.572975   -2.41799   ]
 [-0.440097    0.395058    0.806377    0.00449882]
 [ 0.498562    0.854388   -0.146479   -0.261289  ]
 [ 0.          0.          0.          1.        ]]
BYTES 921600 614400
[[-0.746971  0.33345  -0.575192 -2.42608 ]
 [-0.443189  0.395163  0.80463   0.010116]
 [ 0.495598  0.855954 -0.147394 -0.268958]
 [ 0.        0.        0.        1.      ]]
BYTES 921600 614400
[[-0.754727   0.319426  -0.573023  -2.44337  ]
 [-0.442705   0.396622   0.804179   0.0201067]
 [ 0.484149   0.860615  -0.157929  -0.280052 ]
 [ 0.         0.         0.         1.       ]]
BYTES 921600 614400
[[-0.759128   0.313678  -0.570378  -2.45198  ]
 [-0.441542   0.3957     0.805272   0.0247288]
 [ 0.478294   0.86315   -0.161885  -0.288479 ]
 [ 0.         0.         0.         1.       ]]
BYTES 921600 614400
[[-0.767917   0.300531  -0.565672  -2.4712   ]
 [-0.438703   0.396718   0.806321   0.0326171]
 [ 0.466737   0.86735   -0.172803  -0.300723 ]
 [ 0.         0.       

 99%|█████████▉| 8498/8560 [00:09<00:00, 942.23it/s]

[[-0.880474 -0.318146  0.351495 -2.91062 ]
 [ 0.261531  0.29245   0.919823  0.118026]
 [-0.395432  0.901807 -0.17429  -0.346094]
 [ 0.        0.        0.        1.      ]]
BYTES 921600 614400
[[-0.883371 -0.317708  0.344553 -2.91166 ]
 [ 0.254549  0.292045  0.921908  0.117824]
 [-0.393522  0.902093 -0.177112 -0.344922]
 [ 0.        0.        0.        1.      ]]
BYTES 921600 614400
[[-0.886278 -0.315708  0.338879 -2.91323 ]
 [ 0.248806  0.2926    0.923299  0.117371]
 [-0.390649  0.902615 -0.180775 -0.34609 ]
 [ 0.        0.        0.        1.      ]]
BYTES 921600 614400
[[-0.888005 -0.314231  0.335717 -2.913   ]
 [ 0.245108  0.294258  0.923761  0.116956]
 [-0.389062  0.902591 -0.184282 -0.345056]
 [ 0.        0.        0.        1.      ]]
BYTES 921600 614400
[[-0.891912 -0.312236  0.327112 -2.91067 ]
 [ 0.236481  0.294521  0.925923  0.11572 ]
 [-0.385448  0.903198 -0.188849 -0.345043]
 [ 0.        0.        0.        1.      ]]
BYTES 921600 614400
[[-0.894227 -0.309894  0.322993 -2.

100%|██████████| 8560/8560 [00:09<00:00, 907.38it/s]
C:\Users\unid0\AppData\Local\Temp/ipykernel_4052/632782350.py:121: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.array(data[key], dtype=np.float).tofile(out)


BYTES 921600 614400
[[-0.925825 -0.201112  0.320002 -2.90804 ]
 [ 0.217643  0.408517  0.886423  0.233019]
 [-0.308997  0.890319 -0.334445 -0.52124 ]
 [ 0.        0.        0.        1.      ]]
BYTES 921600 614400
[[-0.925601 -0.203134  0.319374 -2.9106  ]
 [ 0.215152  0.411842  0.885492  0.23408 ]
 [-0.311405  0.888326 -0.337496 -0.521906]
 [ 0.        0.        0.        1.      ]]
BYTES 921600 614400
[[-0.925753 -0.203648  0.318605 -2.91191 ]
 [ 0.214159  0.41204   0.88564   0.234836]
 [-0.311637  0.888116 -0.337834 -0.521934]
 [ 0.        0.        0.        1.      ]]
BYTES 921600 614400
[[-0.925797 -0.205104  0.31754  -2.91488 ]
 [ 0.212155  0.41333   0.885522  0.235185]
 [-0.312873  0.887181 -0.339146 -0.52474 ]
 [ 0.        0.        0.        1.      ]]
BYTES 921600 614400
[[-0.925859 -0.206896  0.316195 -2.91496 ]
 [ 0.210132  0.413557  0.885898  0.235677]
 [-0.314053  0.886659 -0.33942  -0.525293]
 [ 0.        0.        0.        1.      ]]
BYTES 921600 614400
[[-0.925448 -0.

### Custom dataset

In [ ]:
from scipy.spatial.transform import Rotation as rot

def read_frames(folder, obj, light='ambient@best', num_frames=100):
    frames = {'m_colorCompressed': [], 'm_depthCompressed': [], 'm_cameraToWorld': [],
              'm_timeStampColor': [], 'm_timeStampDepth': [], 
              'm_colorSizeBytes': [], 'm_depthSizeBytes': []}
    frames['num_frames'] = num_frames
    folder_frames = folder + '/processed_scans/images/'
    for i in range(int(num_frames)):
        frames['m_colorCompressed'].append(read_image(folder + f'undist/{obj}/tis_right/rgb/{number:04d}.jpg'))
        frames['m_timeStampColor'].append(i)
        frames['m_depthCompressed'].append(read_image(folder + f'depthmaps/{obj}/stl@tis_right.undist/{number:04d}.png'))
        frames['m_timeStampDepth'].append(i)
        
        h, w, c = frames['m_colorCompressed'][-1].shape
        frames['m_colorSizeBytes'].append(h*w*c) # uint8 * h * w * c 
        h, w = frames['m_depthCompressed'][-1].shape
        frames['m_depthSizeBytes'].append(2*h*w) # uint16 * h * w
        
    with open(folder + '/calibration/undist_extrinsics/_calibration/tis_right.images.txt') as f:
        for line in f.readlines():
            parsed_line = line().split(' ')
            transform_matrix = np.ones((4, 4), dtype=np.float32)
            transform_matrix[:3, :3] = rot.from_quat(np.array(parsed_line[1:4], dtype=np.float32)).as_matrix()
            transform_matrix[:. -1] = np.array(parsed_line[4:], dtype=np.float32)
            
            frames['m_cameraToWorld'].append(transform_matrix)
            
    return frames

In [5]:
from PIL import Image

In [16]:
im = np.array(read_image('./0000.png'), dtype=np.uint16)

In [17]:
im.max(), im.min()

(255, 0)

In [1]:
from image_utils import unpack_float32

In [6]:
im_depth = unpack_float32(read_image('./0000.png'))

In [8]:
im_depth[np.isnan(im_depth)] = 0.0

In [9]:
im_depth.max()

0.718291